In [3]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# --- 1. CARICAMENTO DATI ---
# Ora che funzionano, lascia i percorsi così come sono nel tuo ultimo tentativo
df_train = pd.read_csv('train_step6.csv')
df_val   = pd.read_csv('val_step6.csv')

# --- 2. PREPARAZIONE ---
TARGET_COL = 'IS_SEVERE'

X_train = df_train.drop(columns=[TARGET_COL])
y_train = df_train[TARGET_COL]

X_val   = df_val.drop(columns=[TARGET_COL])
y_val   = df_val[TARGET_COL]

print(f"📊 Distribuzione Train Originale: \n{y_train.value_counts()}")

# --- 3. SMOTE VELOCE ---
print("\n⚙️ Applicazione SMOTE Standard...")

# ⚠️ CORREZIONE QUI: Ho rimosso n_jobs=-1
smt = SMOTE(random_state=42)

# Applichiamo
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

print(f"✅ Nuova Distribuzione Train (Bilanciata): \n{y_train_res.value_counts()}")

# --- 4. ADDESTRAMENTO ---
print("\n🚀 Addestramento modello...")
# Qui n_jobs=-1 funziona ancora ed è utile per il RandomForest
model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train_res, y_train_res)

# --- 5. VALIDAZIONE ---
print("\n--- RISULTATI SU VALIDATION SET (Dati Reali) ---")
y_val_pred = model.predict(X_val)

print(classification_report(y_val, y_val_pred))

📊 Distribuzione Train Originale: 
IS_SEVERE
0    467488
1     68882
Name: count, dtype: int64

⚙️ Applicazione SMOTE Standard...
✅ Nuova Distribuzione Train (Bilanciata): 
IS_SEVERE
0    467488
1    467488
Name: count, dtype: int64

🚀 Addestramento modello...

--- RISULTATI SU VALIDATION SET (Dati Reali) ---
              precision    recall  f1-score   support

           0       0.93      0.94      0.94    116872
           1       0.58      0.52      0.55     17221

    accuracy                           0.89    134093
   macro avg       0.76      0.73      0.74    134093
weighted avg       0.89      0.89      0.89    134093



In [1]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, precision_recall_curve

# --- 1. RICARICA PULITA ---
# Ricarichiamo i dati da zero per evitare confusione
df_train = pd.read_csv('train_step6.csv')
df_val   = pd.read_csv('val_step6.csv')

TARGET_COL = 'IS_SEVERE'
X_train = df_train.drop(columns=[TARGET_COL])
y_train = df_train[TARGET_COL]
X_val   = df_val.drop(columns=[TARGET_COL])
y_val   = df_val[TARGET_COL]

# --- 2. SMOTE E TRAINING (Tutto d'un fiato) ---
print("⚙️ Applicazione SMOTE e Training in corso...")
smt = SMOTE(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

# Usiamo un nome variabile NUOVO per essere sicuri: 'model_smote'
model_smote = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model_smote.fit(X_train_res, y_train_res)
print("✅ Modello SMOTE addestrato.")

# --- 3. CALCOLO SOGLIA ---
# Calcoliamo le probabilità sul validation set
y_scores = model_smote.predict_proba(X_val)[:, 1]

# Cerchiamo la soglia per avere Recall > 80%
precisions, recalls, thresholds = precision_recall_curve(y_val, y_scores)
target_recall = 0.80
# Troviamo l'indice più vicino alla recall desiderata
idx = np.argmin(np.abs(recalls - target_recall))
soglia_ottimale = thresholds[idx]

print(f"\n🎯 Soglia Calcolata: {soglia_ottimale:.4f}")
print(f"   Performance attesa -> Precision: {precisions[idx]:.2f}, Recall: {recalls[idx]:.2f}")

# --- 4. VERIFICA FINALE ---
print("\n--- REPORT REALE (Modello SMOTE con Nuova Soglia) ---")
y_pred_new = (y_scores >= soglia_ottimale).astype(int)
print(classification_report(y_val, y_pred_new))

⚙️ Applicazione SMOTE e Training in corso...
✅ Modello SMOTE addestrato.

🎯 Soglia Calcolata: 0.1500
   Performance attesa -> Precision: 0.37, Recall: 0.80

--- REPORT REALE (Modello SMOTE con Nuova Soglia) ---
              precision    recall  f1-score   support

           0       0.96      0.80      0.87    116872
           1       0.37      0.80      0.51     17221

    accuracy                           0.80    134093
   macro avg       0.67      0.80      0.69    134093
weighted avg       0.89      0.80      0.83    134093



In [2]:
import pandas as pd
from imblearn.over_sampling import SMOTE

# --- 1. CARICAMENTO ---
file_input = 'train_step6.csv'
file_output = 'train_step6_SMOTE.csv' # Il nome del nuovo file che creeremo

print(f"📂 Carico {file_input}...")
df = pd.read_csv(file_input)

# Definisci la colonna target
TARGET_COL = 'IS_SEVERE'

# Separa X (Features) e y (Target)
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

print(f"📊 Distribuzione originale:\n{y.value_counts()}")

# --- 2. APPLICAZIONE SMOTE ---
print("\n⚙️ Applicazione SMOTE in corso...")
smt = SMOTE(random_state=42)
X_res, y_res = smt.fit_resample(X, y)

# --- 3. RICOSTRUZIONE DEL DATAFRAME ---
# SMOTE ci restituisce X e y separati. Dobbiamo riunirli in un unico file CSV.

# Creiamo il DataFrame delle features (usando i nomi delle colonne originali)
df_balanced = pd.DataFrame(X_res, columns=X.columns)

# Aggiungiamo la colonna target (IS_SEVERE)
df_balanced[TARGET_COL] = y_res

print(f"✅ Nuova distribuzione bilanciata:\n{df_balanced[TARGET_COL].value_counts()}")

# --- 4. SALVATAGGIO SU DISCO ---
print(f"\n💾 Salvataggio del nuovo file: {file_output}...")
df_balanced.to_csv(file_output, index=False)

print("🎉 Fatto! Ora hai un file csv con le classi perfettamente bilanciate.")

📂 Carico train_step6.csv...
📊 Distribuzione originale:
IS_SEVERE
0    467488
1     68882
Name: count, dtype: int64

⚙️ Applicazione SMOTE in corso...
✅ Nuova distribuzione bilanciata:
IS_SEVERE
0    467488
1    467488
Name: count, dtype: int64

💾 Salvataggio del nuovo file: train_step6_SMOTE.csv...
🎉 Fatto! Ora hai un file csv con le classi perfettamente bilanciate.
